## Step 1: Install Required Libraries
First, you need to install the necessary Python libraries:

In [ ]:
!pip install yt-dlp pytube pandas webvtt-py requests


In [1]:
import os
import requests
import pandas as pd
import webvtt
from pytube import Playlist
import yt_dlp
import re
import time
import unicodedata

# Assume `playlist` is a pytube Playlist object already initialized
playlist_url = 'https://www.youtube.com/playlist?list=PLS3XGZxi7cBUy1RAOyjaecHIM2z19fXO5'
playlist = Playlist(playlist_url)

# Get all video URLs in the playlist
video_urls = [video.watch_url for video in playlist.videos]

# Directory to save thumbnails and transcripts
output_dir = 'data/en_BBCNews'
os.makedirs(output_dir, exist_ok=True)

def sanitize_filename(name):
    """
    Sanitize the filename by removing invalid characters, full-width symbols,
    stripping newlines, and normalizing unicode to ASCII where possible.
    """
    # Normalize full-width characters to their ASCII equivalents
    name = unicodedata.normalize('NFKC', name)  # Normalize Unicode characters to ASCII-compatible form
    
    # Remove problematic characters for filenames
    name = re.sub(r'[\\/*?:"<>|]', "", name)  # Remove invalid characters
    name = name.replace('\n', '').replace('\r', '')  # Remove newlines
    name = name.strip()  # Trim leading/trailing spaces

    # Ensure the filename length is not too long (you can set a specific limit)
    max_filename_length = 255
    return name[:max_filename_length]

def download_thumbnail(thumbnail_url, title):
    try:
        response = requests.get(thumbnail_url)
        sanitized_title = sanitize_filename(title)
        thumbnail_file = os.path.join(output_dir, f'{sanitized_title}.jpg')
        with open(thumbnail_file, 'wb') as file:
            file.write(response.content)
        return thumbnail_file
    except Exception as e:
        print(f"Failed to download thumbnail for {title}: {e}")
        return None

def download_subtitles(video_url, video_title, subtitle_languages):
    sanitized_title = sanitize_filename(video_title)
    
    # Define the download options
    ydl_opts = {
        'writesubtitles': True,
        'writeautomaticsub': True,
        'skip_download': True,  # Skip video download
        'outtmpl': os.path.join(output_dir, f'{sanitized_title}.%(ext)s'),  # Use sanitized title
        'restrictfilenames': True,
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            if 'en' in subtitle_languages:
                # Download English subtitles
                subtitle_file = os.path.join(output_dir, f'{sanitized_title}.en.vtt')
                if not os.path.exists(subtitle_file):
                    ydl_opts['subtitleslangs'] = ['en']
                    ydl.download([video_url])  # Download subtitles for the video
                return subtitle_file
            elif subtitle_languages:
                # Download the first available subtitle if English is not found
                first_lang = list(subtitle_languages)[0]
                subtitle_file = os.path.join(output_dir, f'{sanitized_title}.{first_lang}.vtt')
                if not os.path.exists(subtitle_file):
                    ydl_opts['subtitleslangs'] = [first_lang]
                    ydl.download([video_url])
                return subtitle_file
                
    except yt_dlp.utils.DownloadError as e:
        error_message = str(e)
        print(f"Error downloading subtitles for {video_title}: {error_message}")
        return None
                
def download_video_info(video_url, max_retries=3, retry_interval=180):
    ydl_opts = {
        'writesubtitles': True,
        'writeautomaticsub': True,
        'skip_download': True,
        #'outtmpl': os.path.join(output_dir, f'{"%(sanitized_title)s"}.%(ext)s'),
        'postprocessors': [{
            'key': 'FFmpegMetadata',
        }],
    }
    retries = 0
    while retries < max_retries:
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info_dict = ydl.extract_info(video_url, download=False)
                video_title = info_dict.get('title', None)
                video_id = info_dict.get('id', None)
                thumbnail_url = info_dict.get('thumbnail', None)
                publish_date = info_dict.get('upload_date', None)  # Format: 'YYYYMMDD'
                subtitle_languages = info_dict.get('automatic_captions', {}).keys()

                # Download thumbnail
                thumbnail_file = None
                if thumbnail_url:
                    thumbnail_file = download_thumbnail(thumbnail_url, video_title)

                # Download subtitles
                subtitle_file = None
                subtitle_file = download_subtitles(video_url, video_title, subtitle_languages)
                
            # Return the collected video information
            return video_id, video_title, thumbnail_file, subtitle_file, publish_date

        except yt_dlp.utils.DownloadError as e:
            error_message = str(e)
            if "Sign in to confirm you’re not a bot" in error_message:
                retries += 1
                print(f"Error: {error_message}. Retrying in {retry_interval} seconds... (Attempt {retries}/{max_retries})")
                time.sleep(retry_interval)  # Wait before retrying
            else:
                print(f"Failed to download video info for {video_url}: {e}")
                return None, None, None, None, None

    # After max retries, log the failure and return None
    print(f"Max retries reached. Skipping video: {video_url}")
    return None, None, None, None, None

def vtt_to_text(vtt_file_path):
    try:
        text = ''
        lines = []
        deduplicated_lines = []
        for caption in webvtt.read(vtt_file_path):
            # Split the block into lines and deduplicate them within the block
            lines.extend(caption.text.strip().split('\n'))
            #print(caption.text.strip().split('\n'))
        for line in lines:
            if line and (not deduplicated_lines or line != deduplicated_lines[-1]):  # Avoid consecutive duplicates
                deduplicated_lines.append(line)
     
        text = "\n".join(deduplicated_lines)
    
    except Exception as e:
        print(f"Failed to convert {vtt_file_path} to text: {e}")
        
    return text

# Create a list to store video information
data = []

# Loop through each video URL and download the required data
for video_url in video_urls:
    video_id, video_title, thumbnail_file, subtitle_file, publish_date = download_video_info(video_url)
    # If video info is successfully retrieved
    if video_id and video_title:
        # Convert subtitles to text if available
        transcript = None
        if subtitle_file and os.path.exists(subtitle_file):
            transcript = vtt_to_text(subtitle_file)
        else:
            print("subtitle file not found")
        # Append data to the list
        data.append({
            'video_id': video_id,
            'video_title': video_title,
            'thumbnail_path': thumbnail_file,
            'transcript': transcript,
            'publish_date': publish_date
        })

# Convert the list to a DataFrame
df = pd.DataFrame(data, columns=['video_id', 'video_title', 'thumbnail_path', 'transcript', 'publish_date'])

# Save the DataFrame to a CSV file
df.to_csv(os.path.join(output_dir,"../", 'en_BBCNews.csv'), index=False)

print(f"Dataset created successfully with {len(df)} entries.")

[youtube] Extracting URL: https://youtube.com/watch?v=93WDYXl8658
[youtube] 93WDYXl8658: Downloading webpage
[youtube] 93WDYXl8658: Downloading ios player API JSON
[youtube] 93WDYXl8658: Downloading web creator player API JSON
[youtube] 93WDYXl8658: Downloading m3u8 information
[info] 93WDYXl8658: Downloading subtitles: en


[youtube] Extracting URL: https://youtube.com/watch?v=nzeRnUHmPwM
[youtube] nzeRnUHmPwM: Downloading webpage
[youtube] nzeRnUHmPwM: Downloading ios player API JSON
[youtube] nzeRnUHmPwM: Downloading web creator player API JSON
[youtube] nzeRnUHmPwM: Downloading m3u8 information
[info] nzeRnUHmPwM: Downloading subtitles: en


[youtube] Extracting URL: https://youtube.com/watch?v=vXrlq7A18S4
[youtube] vXrlq7A18S4: Downloading webpage
[youtube] vXrlq7A18S4: Downloading ios player API JSON
[youtube] vXrlq7A18S4: Downloading web creator player API JSON
[youtube] vXrlq7A18S4: Downloading m3u8 information
[info] vXrlq7A18S4: Downloading subtitles: en


[youtube] Extracting URL: https://youtube.com/watch?v=pNEb-dY3tRY
[youtube] pNEb-dY3tRY: Downloading webpage
[youtube] pNEb-dY3tRY: Downloading ios player API JSON
[youtube] pNEb-dY3tRY: Downloading web creator player API JSON
[youtube] pNEb-dY3tRY: Downloading m3u8 information
[info] pNEb-dY3tRY: Downloading subtitles: en


[youtube] Extracting URL: https://youtube.com/watch?v=w1V9YrsK9QY
[youtube] w1V9YrsK9QY: Downloading webpage
[youtube] w1V9YrsK9QY: Downloading ios player API JSON
[youtube] w1V9YrsK9QY: Downloading web creator player API JSON
[youtube] w1V9YrsK9QY: Downloading m3u8 information
[info] w1V9YrsK9QY: Downloading subtitles: en


...

[info] IIJ2S6S2B2o: Downloading 1 format(s): 18
[info] Writing video subtitles to: data\en_BBCNews\UN says Gaza 'on knife edge' - BBC News.en.vtt
[download] Destination: data\en_BBCNews\UN says Gaza 'on knife edge' - BBC News.en.vtt
[download] 100% of   18.59KiB in 00:00:00 at 130.33KiB/s
[youtube] Extracting URL: https://youtube.com/watch?v=aqDq6UkeAaw
[youtube] aqDq6UkeAaw: Downloading webpage
[youtube] aqDq6UkeAaw: Downloading ios player API JSON
[youtube] aqDq6UkeAaw: Downloading web creator player API JSON
[youtube] aqDq6UkeAaw: Downloading m3u8 information
[info] aqDq6UkeAaw: Downloading subtitles: en


[youtube] Extracting URL: https://youtube.com/watch?v=aqDq6UkeAaw
[youtube] aqDq6UkeAaw: Downloading webpage
[youtube] aqDq6UkeAaw: Downloading ios player API JSON
[youtube] aqDq6UkeAaw: Downloading web creator player API JSON
[youtube] aqDq6UkeAaw: Downloading m3u8 information
[info] aqDq6UkeAaw: Downloading subtitles: en


[info] aqDq6UkeAaw: Downloading 1 format(s): 18
[info] Writing video subtitles to: data\en_BBCNews\Where are Palestinian rockets being fired towards BBC News.en.vtt
[download] Destination: data\en_BBCNews\Where are Palestinian rockets being fired towards BBC News.en.vtt
[download] 100% of    7.73KiB in 00:00:00 at 67.55KiB/s
[youtube] Extracting URL: https://youtube.com/watch?v=2rF71YbGn3o
[youtube] 2rF71YbGn3o: Downloading webpage
[youtube] 2rF71YbGn3o: Downloading ios player API JSON
[youtube] 2rF71YbGn3o: Downloading web creator player API JSON
[youtube] 2rF71YbGn3o: Downloading m3u8 information
[info] 2rF71YbGn3o: Downloading subtitles: en


[youtube] Extracting URL: https://youtube.com/watch?v=2rF71YbGn3o
[youtube] 2rF71YbGn3o: Downloading webpage
[youtube] 2rF71YbGn3o: Downloading ios player API JSON
[youtube] 2rF71YbGn3o: Downloading web creator player API JSON
[youtube] 2rF71YbGn3o: Downloading m3u8 information
[info] 2rF71YbGn3o: Downloading subtitles: en


[info] 2rF71YbGn3o: Downloading 1 format(s): 18
[info] Writing video subtitles to: data\en_BBCNews\Video shows Israeli airstrikes on Gaza Strip - BBC News.en.vtt
[download] Destination: data\en_BBCNews\Video shows Israeli airstrikes on Gaza Strip - BBC News.en.vtt
[download] 100% of    7.61KiB in 00:00:00 at 54.88KiB/s
Dataset created successfully with 551 entries.
